# Ayudantía Serialización de objetos
#### Gabriel Lyon y José Manuel Larraín

### Funciones útiles:

* dump(s)
* load(s)

#### Pickle
* \_\_getstate\_\_
* \_\_setstate\_\_

#### JSON
* JSONEncoder
* object_hook


## Introducción

¡Los alumnos del DDC tiene la idea del siglo! El servicio de mensajería instantánea DDChat, el que
permitirá comunicarse de forma confidencial entre los alumnos del DDC. Lamentablemente, el infame Dr. Mavrakis, logró acceder a la base de datos y leer todos los mensajes enviados por los alumnos.

Su misión es aplicar un sistema de seguridad encriptado para que no se vuelva a repetir tal desgracia,
pero como usted no se quiere quedar sin las últimas copuchas del DDC, implementará un sistema que le
permita visualizar las conversaciones entre dos alumnos.

### Formato Datos

#### Mensajes

Los mensajes vienen en el siguiente formato:

- {"send_to": 95521376, "content": "No puedo, tengo que estudiar", "send_by": 76251142, "last_view_date": "", "date": "18-4-2017 15:45"}

#### Usuarios

Los contactos vienen en el siguiente formato:

- {"name": "Francisca Rios", "contacts": [], "phone_number": 45348826}


### Requerimientos 

En primer lugar, deberá leer los archivos y transformar su información a objetos de las clases `Usuario` y
`Mensaje`. Esta información está contenida en la carpeta **db**, en donde la carpeta **usr** contiene a los usuarios y la carpeta **msg** contiene los mensajes. Ambos están en formato JSON.

Para lograr este paso deben crear dos funciones que retornen una lista conteniendo sus respectivos objetos.

In [12]:
import pickle
import json
import os
from random import randint
from datetime import datetime


class Usuario:
    def __init__(self, name, contacts, phone_number):
        self.name = name
        self.phone_number = phone_number
        self.contacts = contacts


class Mensaje:
    def __init__(self, send_to, content, send_by, date, last_view_date):
        self.send_to = send_to
        self.content = content
        self.send_by = send_by
        self.last_view_date = last_view_date
        self.date = date
    
    
    def __getstate__(self):
        dict_mensaje = self.__dict__.copy()
        dict_mensaje["content"] =  caesarCipher(dict_mensaje["content"],
                                                dict_mensaje["send_by"])
        
        return dict_mensaje
        
    def __setstate__(self, diccionario):
        d = datetime.now()
        diccionario["last_view_date"] = "{}/{}/{}-{}:{}".format(d.year, d.month, d.day, d.hour, d.minute)
        self.__dict__ = diccionario
        
        
        
def readUsers():
    users = []
    for path in os.listdir("db/usr"):
        with open("db/usr/" + path, "r") as f:
            user = json.load(f, object_hook=lambda dict_obj: Usuario(**dict_obj))
        users.append(user)
    print(users)
    return users
def readMessages():
    mensajes = []
    for path in os.listdir("db/msg"):
        with open ("db/msg/"+ path, "r") as f:
            msg = json.load(f, object_hook=lambda dict_obj: Mensaje(**dict_obj))
        mensajes.append(msg)
    
    return mensajes



Una vez cargado todos los datos, deberá agregar los contactos de cada usuario. Estos cuentan con el atributo `contacts`, que viene vacío. Para que un usuario **x** sea contacto de otro **y**, es necesario que **y** haya enviado un mensaje a **x**, pero **x** no necesariamente tiene a **y** como contacto.

In [13]:
def updateUsers(usarios, mensajes):
    users = {user.phone: user for user in usuarios}
    for mensaje in mensajes:
        u_envio = mensaje.send_by
        u_recive = mensaje.send_to
        if u_recive not in users[u_envio].contacts:
            users[u_envio].contacts.append(u_recive)



Luego deberá guardar los Usuarios actualizados en la carpeta **usr** de la carpeta **secure_db**, los cuales
deben quedar en formato JSON, con un archivo por usuario.

In [14]:
def saveUsers(usuarios):
    
    for usuario in usuarios:
        path = "secure_db/usr/{}".format(str(usuario.phone_number))
        with open(path, "w", encoding="utf-8") as f:
            json.dump(usuario.__dict__, f)
    


En el caso de los mensajes, que se guardan en la carpeta **msg** de **secure_db** utilizando pickle, deberá encriptar su contenido antes de guardarlos (un archivo por mensaje) con el algoritmo de seguridad más novedoso y seguro del momento: **Caesar Cipher**.

**Caesar Cipher** es muy simple; dado un string y un número n, cada uno de los caracteres se reemplazarán
por el n-ésimo caracter siguiente. Por ejemplo, dado n = 2 y la palabra Cesar, se obtendrá Eguct. El código
Cesar se resume en la siguiente fórmula: `y(x, n) = (x+n) mód 26`, donde cada letra **x** está asociada a un número y **n** son los números a desplazar **x**, e **y** es el nuevo valor del caracer. Una operación modular retorna el resto de la división. Por ejemplo (5 + 4) mód 2 = 1. Asuman que el alfabeto no incluye a la ñ ni caracteres acentuados.

¡La encriptación debe tomar lugar justo antes de serializar el archivo! La llave **n** a utilizar corresponderá
al número telefónico de quién envió el mensaje.

Finalmente, debe quedar registrado cada vez que alguien deserialice un mensaje. Para esto, deben actualizar en el archivo respectivo el atributo `last_view_date` con la fecha y hora de la última deserialización de un mensaje.

In [15]:
def saveEncriptedMessages(mensajes):
    for msg in mensajes:
        #el randint es solo pa crear el nombre del archivo en donde se va a guardar
        path = "secure_db/msg/{}".format(str(randint(0, 99999999)))
        with open (path, "wb") as f:
            picke.dump(msg, f)
        
        
        
def caesarCipher(string, key):
    new = ""
    for caracter in string:
        if caracter.isalpha():
            #97 = orden de la a
            ascii_value = ord(caracter) - 97
            value = ascii_value + key
            value = value % 26
            nuevo_caracter = chr(value + 97)
            
            new += nuevo_caracter
    
    return new